In [14]:
from arcgis.gis import GIS
from data_engineering.utils import fetch_charging_stations, fetch_traffic_accidents
from urban_traffic.utils import fetch_traffic_data, read_traffic_features
import os
import pandas as pd

### Additional dependencies
We load the necessary variables using `dotenv`.  
Please, ensure that you setup before: `pip install python-dotenv`  
Furthermore, put a `.env` file into the root of this repository.

In [2]:
from dotenv import load_dotenv

def load_vars():
    load_dotenv(dotenv_path="../../../../.env")
    return os.getenv("ARCGIS_API_KEY"), os.getenv("TRAFFIC_DATA_FILE"), os.getenv("TRAFFIC_FEATURES")

In [3]:
api_key, traffic_data_filepath, traffic_features_filepath = load_vars()

### Connect to your GIS portal

In [4]:
gis = GIS(api_key=api_key)

In [5]:
map_view = gis.map("Frankfurt am Main, Germany")
map_view.basemap.basemap = "osm"
map_view.zoom = 14
map_view

Map(center=[6465500.493115716, 966524.0204067213], extent={'xmin': 951495.8891496294, 'ymin': 6442099.72909625…

In [25]:
distance_in_meters = 50
lon, lat = map_view.center[:2]
traffic_features_sdf = read_traffic_features(traffic_features_filepath, lon=lon, lat=lat, meters=distance_in_meters)
traffic_features_sdf[["trip", "person", "vehicle_type"]]

,trip,person,vehicle_type
0,1640,1018,Car
1,1640,1018,Car
2,1640,1018,Car
3,1640,1018,Car
4,1640,1018,Car
...,...,...,...
1741,294,186,None
1742,294,186,None
1743,294,186,None
1744,294,186,None


In [11]:
traffic_features_sdf.dtypes

OBJECTID           int64
SHAPE           geometry
trip               int64
person             int64
vehicle_type      object
trip_time         object
dtype: object

In [27]:
import pandas as pd

def prepare_traffic(traffic_df: pd.DataFrame) -> pd.DataFrame:
    # Extract hours, minutes, and seconds
    traffic_df["trip_time"] = pd.to_datetime(traffic_df["trip_time"], format="%Y-%m-%dT%H:%M:%S")
    traffic_df["hour"] = traffic_df["trip_time"].dt.strftime("%H")
    traffic_df["minute"] = traffic_df["trip_time"].dt.strftime("%M")
    traffic_df["second"] = traffic_df["trip_time"].dt.strftime("%S")
    traffic_df = traffic_df.drop(columns=["trip_time"], axis=1)
    traffic_df = traffic_df.fillna({"vehicle_type": "Pedestrian"})
    return traffic_df

In [28]:
prepared_traffic_features_sdf = prepare_traffic(traffic_features_sdf.copy())
prepared_traffic_features_sdf

,OBJECTID,SHAPE,trip,person,vehicle_type,hour,minute,second
0,2308,"{""x"": 8.689174832000049, ""y"": 50.1072080680000...",1640,1018,Car,00,17,49
1,2313,"{""x"": 8.688980546000039, ""y"": 50.1072080240000...",1640,1018,Car,00,17,50
2,2316,"{""x"": 8.688791012000024, ""y"": 50.1072204760000...",1640,1018,Car,00,17,51
3,2320,"{""x"": 8.688596634000021, ""y"": 50.1072198200000...",1640,1018,Car,00,17,52
4,2324,"{""x"": 8.688402256000074, ""y"": 50.1072191630000...",1640,1018,Car,00,17,53
...,...,...,...,...,...,...,...,...
1741,16745060,"{""x"": 8.687910028000033, ""y"": 50.1070745070000...",294,186,Pedestrian,20,55,41
1742,16745153,"{""x"": 8.687905365000063, ""y"": 50.1070859010000...",294,186,Pedestrian,20,55,42
1743,16745246,"{""x"": 8.687891322000041, ""y"": 50.1070840100000...",294,186,Pedestrian,20,55,43
1744,16745339,"{""x"": 8.687878086000069, ""y"": 50.1070837100000...",294,186,Pedestrian,20,55,44


In [13]:
traffic_features_sdf

,OBJECTID,SHAPE,trip,person,vehicle_type,trip_time
0,197576,"{""x"": 8.662648736000051, ""y"": 50.1075762820000...",9568,5966,None,2023-11-21 03:31:24
1,197601,"{""x"": 8.662654839000027, ""y"": 50.1075655800000...",9568,5966,None,2023-11-21 03:31:25
2,197626,"{""x"": 8.66266392700004, ""y"": 50.10755605600008...",9568,5966,None,2023-11-21 03:31:26
3,197651,"{""x"": 8.662673014000063, ""y"": 50.1075465310000...",9568,5966,None,2023-11-21 03:31:27
4,197676,"{""x"": 8.66268210100003, ""y"": 50.10753700500004...",9568,5966,None,2023-11-21 03:31:28
...,...,...,...,...,...,...
6514,17221245,"{""x"": 8.66334114600005, ""y"": 50.10684543900004...",6263,3892,None,2023-11-21 23:28:15
6515,17221270,"{""x"": 8.663349032000042, ""y"": 50.1068371680000...",6263,3892,None,2023-11-21 23:28:16
6516,17221295,"{""x"": 8.663356917000044, ""y"": 50.1068288970000...",6263,3892,None,2023-11-21 23:28:17
6517,17221320,"{""x"": 8.663364804000025, ""y"": 50.1068206260000...",6263,3892,None,2023-11-21 23:28:18
